In [1]:
## ATTENTION: BEVOR RUNNING PLEASE SELECT A NEW runNAME - check register!!!!!
import os
os.chdir("/mnt/storage1/deep_contact_luca/DP2500k/codes")
dir2save = "/mnt/storage1/deep_contact_luca/DP2500k/results_logs/"
######### SELECT runNAME ####################
runName = "dp2500-01/24"
#############################################

import util
import numpy as np
import pandas as pd
from keras import metrics
from keras.models import Model
from keras.layers import (
    concatenate, Dense, LSTM, Input, Convolution1D,
    MaxPooling1D, BatchNormalization, Dropout,
    Bidirectional, Flatten, CuDNNLSTM
)
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import (
    ModelCheckpoint, EarlyStopping, CSVLogger
)
from tensorflow.keras import backend as K
import tensorflow as tf
import h5py
import sklearn.preprocessing as skprep
import csv
import re
import subprocess
from datetime import datetime

2024-04-21 09:37:35.640848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 09:37:35.694388: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 09:37:36.021116: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2024-04-21 09:37:36.021147: W tensorflow/com

In [2]:
np.set_printoptions(suppress=True)
tf.keras.backend.clear_session()

# Configuration values
data_dir = "/mnt/storage1/deep_contact_luca/DP2500k/data"
data_samples_to_use = 202333
HiChip_marks = "k4me_2500,k27me_2500"
#percent_in_test_set = 10
two_neurons_output = False
HDF5_samples_num = 202333
ATAC_based_negative_samples = False
using_histones = False
using_H3 = False
using_ATAC = False
using_fasta_seq = True
using_meth_info = False

num_fasta_seq_rows = 4 if using_fasta_seq else 0

num_seq_signals_excluding_histones = using_H3 + using_ATAC + num_fasta_seq_rows + using_meth_info

In [3]:
def specificity(y_true, y_pred):    
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [5]:
def build_model():
    # Hyperparameters
    num_filters = [256, 512]
    kernel_sizes = [64, 32, 16]
    pool_sizes = [4, 4, 4]
    stride_sizes = [2, 2, 2]
    lstm_units = [256, 256]
    dropout_rate = 0.5
    dense_units = 512
    dense_activation = 'tanh'
    dense_dropout_rate = 0.3
    learning_rate = 1e-4

    # Input layers
    input1 = Input(shape=(2500, 4))
    input2 = Input(shape=(2500, 4))

    # Convolutional layers and pooling for the first sequence
    conv1a = Convolution1D(filters=num_filters[0], kernel_size=kernel_sizes[0], padding="valid", activation='relu',
                           kernel_regularizer=l2(1e-3))(input1)
    pool1a = MaxPooling1D(pool_size=pool_sizes[0], strides=stride_sizes[0])(conv1a)
    conv2a = Convolution1D(filters=num_filters[1], kernel_size=kernel_sizes[1], padding="valid", activation='relu',
                           kernel_regularizer=l2(1e-3))(pool1a)
    pool2a = MaxPooling1D(pool_size=pool_sizes[1], strides=stride_sizes[1])(conv2a)

    # Convolutional layers and pooling for the second sequence
    conv1b = Convolution1D(filters=num_filters[0], kernel_size=kernel_sizes[0], padding="valid", activation='relu',
                           kernel_regularizer=l2(1e-3))(input2)
    pool1b = MaxPooling1D(pool_size=pool_sizes[0], strides=stride_sizes[0])(conv1b)
    conv2b = Convolution1D(filters=num_filters[1], kernel_size=kernel_sizes[1], padding="valid", activation='relu',
                           kernel_regularizer=l2(1e-3))(pool1b)
    pool2b = MaxPooling1D(pool_size=pool_sizes[1], strides=stride_sizes[1])(conv2b)

    # Merge the outputs
    merged = concatenate([pool2a, pool2b], axis=1)
    dropout = Dropout(dropout_rate)(merged)

    # Bidirectional LSTM layer
    lstm = Bidirectional(CuDNNLSTM(units=lstm_units[0]))(dropout)
    flatten = Flatten()(lstm)

    # Dense layers
    dense = Dense(dense_units, activation=dense_activation)(flatten)
    batch_norm = BatchNormalization()(dense)
    dropout2 = Dropout(dense_dropout_rate)(batch_norm)

    # Output layer
    output = Dense(units=1, activation='sigmoid')(dropout2)

    # Create the model
    model = Model(inputs=[input1, input2], outputs=output, name='GenomicLinks')
    
    # Compile the model - add more metrics
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate),
                  metrics=[metrics.SpecificityAtSensitivity(0.5), 'accuracy', specificity])
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', specificity])

    model.summary()
    return model

model = build_model()


Model: "GenomicLinks"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 2500, 4)]    0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 2500, 4)]    0           []                               
                                                                                                  
 conv1d_4 (Conv1D)              (None, 2437, 256)    65792       ['input_3[0][0]']                
                                                                                                  
 conv1d_6 (Conv1D)              (None, 2437, 256)    65792       ['input_4[0][0]']                
                                                                                       

In [5]:
def start_dp():
    h5_filename = "/mnt/storage1/defne_dc/data/regions_labels_rand_neg_202344.h5"
    num_seq_signals = 4
    train_r1, train_r2, train_labels, valid_r1, valid_r2, valid_labels, test_r1, test_r2, test_labels = util.data_to_train_test(h5_filename, data_samples_to_use)
    return train_r1, train_r2, train_labels, valid_r1, valid_r2, valid_labels, test_r1, test_r2, test_labels

# Now this function returns 9 variables instead of 3 or 6.
train_r1, train_r2, train_labels, valid_r1, valid_r2, valid_labels, test_r1, test_r2, test_labels = start_dp()


es = EarlyStopping(monitor='val_specificity_at_sensitivity',
                   patience=10,
                   verbose=1,
                   mode='max')


start reading h5
start_time_start_reading_H5
--- 1.1920928955078125e-06 seconds ---
reading H5 file done
--- 12.603403806686401 seconds ---


In [6]:
log_filename = dir2save + 'log_' + runName + '.csv'
weights_filename = dir2save + 'weights-improvement-' + runName + '-{epoch:02d}.hdf5'

csv_logger = CSVLogger(log_filename, append=True, separator=';')
checkpoint = ModelCheckpoint(weights_filename, monitor='val_specificity_at_sensitivity', verbose=1,
                             save_best_only=True, mode='max')
callbacks_list = [checkpoint, csv_logger, es]


In [7]:
x_train = [np.asarray(train_r1, dtype=np.uint8), np.asarray(train_r2, dtype=np.uint8)]
y_train = np.asarray(train_labels, dtype=np.uint8)

# Update this part to use valid_r1, valid_r2, and valid_labels instead of test_r1, test_r2, and test_labels
x_val = [np.asarray(valid_r1, dtype=np.uint8), np.asarray(valid_r2, dtype=np.uint8)]
y_val = np.asarray(valid_labels, dtype=np.uint8)


In [8]:
num_epochs = 30
batch_size = 100
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    verbose=1,
    callbacks=callbacks_list,
    shuffle=True,
    validation_data=(x_val, y_val)
)


Epoch 1/30


2023-09-14 11:56:40.627885: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8801
2023-09-14 11:56:41.762291: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1619/1619 [==============================] - ETA: 0s - loss: 1.0267 - specificity_at_sensitivity: 0.4889 - accuracy: 0.5081 - specificity: 0.5137
Epoch 1: val_specificity_at_sensitivity improved from -inf to 0.54878, saving model to /mnt/storage1/deep_contact_luca/DP2500k/results_logs/weights-improvement-dp2500-1.1.09.23-01.hdf5
1619/1619 [==============================] - 341s 209ms/step - loss: 1.0267 - specificity_at_sensitivity: 0.4889 - accuracy: 0.5081 - specificity: 0.5137 - val_loss: 0.8078 - val_specificity_at_sensitivity: 0.5488 - val_accuracy: 0.5305 - val_specificity: 0.6794
Epoch 2/30
1619/1619 [==============================] - ETA: 0s - loss: 0.7391 - specificity_at_sensitivity: 0.6826 - accuracy: 0.5974 - specificity: 0.5990
Epoch 2: val_specificity_at_sensitivity improved from 0.54878 to 0.80032, saving model to /mnt/storage1/deep_contact_luca/DP2500k/results_logs/weights-improvement-dp2500-1.1.09.23-02.hdf5
1619/1619 [==============================] - 335s 207ms/step 

In [21]:
def predict(model, test_r1, test_r2, test_labels):
    y_pred = model.predict([test_r1, test_r2]).flatten()  # Flatten to make it a 1D array
    y_true = test_labels

    # Debugging info
    print("y_pred shape:", y_pred.shape)
    print("y_true shape:", y_true.shape)
    print("Unique values in y_pred:", np.unique(np.round(y_pred)))
    print("Unique values in y_true:", np.unique(y_true))
    
    y_pred_rounded = np.round(y_pred)
    
    TP = np.sum((y_true == y_pred_rounded) & (y_true == 1))
    FP = np.sum((y_pred_rounded == 1) & (y_true != y_pred_rounded))
    TN = np.sum((y_true == y_pred_rounded) & (y_true == 0))
    FN = np.sum((y_pred_rounded == 0) & (y_true != y_pred_rounded))
    
    return TP, FP, TN, FN, y_pred, y_true

TP, FP, TN, FN, y_pred, y_true = predict(model, test_r1, test_r2, test_labels)
print(TP, FP, TN, FN)



633/633 [==============================] - 24s 38ms/step
y_pred shape: (20233,)
y_true shape: (20233,)
Unique values in y_pred: [0. 1.]
Unique values in y_true: [0 1]
8602 1590 8562 1479


In [18]:
# Calculate Accuracy
accuracy = round((TP + TN) / (TP + TN + FP + FN), 3)

# Calculate Precision
precision = round(TP / (TP + FP), 3)

# Calculate Recall/Sensitivity
recall = round(TP / (TP + FN), 3)

# Calculate F1 Score
f1_score = round(2 * (precision * recall) / (precision + recall), 3)

# Calculate Specificity
specificity = round(TN / (TN + FP), 3)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")
print(f"Specificity: {specificity}")



Accuracy: 0.848
Precision: 0.844
Recall: 0.853
F1 Score: 0.848
Specificity: 0.843


In [22]:
from sklearn.metrics import roc_auc_score, average_precision_score

# y_pred should contain the predicted probabilities
# y_true should be the true binary labels

# Calculate ROC AUC
roc_auc = round(roc_auc_score(y_true, y_pred), 3)

# Calculate AUPRC
auprc = round(average_precision_score(y_true, y_pred), 3)

print(f"ROC AUC: {roc_auc}")
print(f"AUPRC: {auprc}")


ROC AUC: 0.925
AUPRC: 0.926
